# Прогресс

* `[v]` Упражнение 3-4  - удаление бесполезных символов
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики


* `[ ]` Упражнение 5-6  - преобразование к неукорачивающей грамматике
    * ???


* `[v]` Упражнение 7-8  - удаление цепных правил
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_цепных_правил_из_грамматики


* `[ ]` Упражнение 9-10 - исключение левой рекурсии из свободной грамматики
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики
    * https://neerc.ifmo.ru/wiki/index.php?title=Устранение_левой_рекурсии


* `[ ]` Упражнение 11-12 - приведение к нормальной форме Хомского
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_длинных_правил_из_грамматики
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_цепных_правил_из_грамматики 
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики
    * https://neerc.ifmo.ru/wiki/index.php?title=Нормальная_форма_Хомского

Еще литература:

* Удаление eps-правил
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики 

* Удаление длинных правил
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_длинных_правил_из_грамматики

In [1]:
import re

In [2]:
def all_terminals(g):
    return set([q for x in g for q in re.findall(r'[^A-Z\s(\->)\|]', x)])

def all_non_terminals(g):
    return set([q for x in g for q in re.findall(r'[A-Z]', x)])

In [3]:
def union_lists(a, b):
    return list(set(a).union(set(b)))

def is_subset_of(a, b):
    return [x for x in a if x in b] == a

In [4]:
def left(r):
    return r.split('->')[0].strip()

def right(r):
    return r.split('->')[1].strip()

In [5]:
def _non_terminals(r, right=True):
    i = int(right)
    return [x for x in re.findall(r'[A-Z]', r.split('->')[i])]

def _terminals(r, right=True):
    i = int(right)
    return [x for x in re.findall(r'[^A-Z\s(\->)\|]', r.split('->')[i])]

def right_non_terminals(r):
    return _non_terminals(r)

def right_terminals(r):
    return _terminals(r)

def left_non_terminals(r):
    return _non_terminals(r, right=False)

def left_terminals(r):
    return _terminals(r, right=False)

In [6]:
def explode(r):
    left, right = r.split('->')
    right = right.split('|')
    return [(left + '->' + p).strip() \
               for p in right]

def explode_grammar(g):
    return [x for r in g for x in explode(r)]

def compact_grammar(g):
    from collections import defaultdict
    dict_ = defaultdict(list)
    for r in g:
        left, right = r.split('->')
        left, right = left.strip(), right.strip()
        dict_[left] += [right]
    return [k + ' -> ' + ' | '.join(v) for k,v in dict_.items()]

# Удаление непорождающих правил

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B5%D1%81%D0%BF%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D1%8B%D1%85_%D1%81%D0%B8%D0%BC%D0%B2%D0%BE%D0%BB%D0%BE%D0%B2_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

In [7]:
g = [
    'S -> AS | BS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [8]:
def delete_non_generating_rules(g):
    g_ = explode_grammar(g)
    Q = []
    for r in g_:
        if len(right_non_terminals(r)) == 0:
            Q = union_lists(Q, left_non_terminals(r)) 
    changed = True
    while changed:
        changed = False
        for r in g_:
            if is_subset_of(right_non_terminals(r), Q): 
                old = len(Q)
                Q = union_lists(Q, left_non_terminals(r))
                if old != len(Q):
                    changed = True
    return compact_grammar([r for r in g_ if is_subset_of(right_non_terminals(r), Q)])

In [9]:
new_grammar = delete_non_generating_rules(g)

In [10]:
for r in new_grammar:
    print(r)

S -> AS | s
E -> EF | FF
A -> a
F -> f


# Удаление недостижимых нетерминалов

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B5%D1%81%D0%BF%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D1%8B%D1%85_%D1%81%D0%B8%D0%BC%D0%B2%D0%BE%D0%BB%D0%BE%D0%B2_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

Нетерминал $A$ называется достижимым (англ. reachable) в КС-грамматике $\Gamma$, если существует порождение $S \Rightarrow^* \alpha A \beta$. Иначе он называется недостижимым (англ. unreachable).

Очевидно, что если нетерминал в левой части правила является достижимым, то и все нетерминалы правой части являются достижимыми.

Алгоритм

* __Шаг 0.__ Множество достижимых нетерминалов состоит из единственного элемента: $\lbrace S \rbrace$.
* __Шаг 1.__ Если найдено правило, в левой части которого стоит нетерминал, содержащийся в множестве, добавим в множество все нетерминалы из правой части.
* __Шаг 2.__ Повторим предыдущий шаг, если множество порождающих нетерминалов изменилось.

Получаем множество всех достижимых нетерминалов, а нетерминалы, не попавшие в него, являются недостижимыми.

$S\rightarrow AS|s \\     E\rightarrow EF|FF \\     A\rightarrow a \\     F\rightarrow f$

In [11]:
g = [
    'S -> AS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [12]:
def delete_unreachable_non_terminals(g):
    g_ = explode_grammar(g)
    Q = ['S']
    changed = True
    while changed:
        changed = False
        for r in g_:
            lnt = left_non_terminals(r).pop()
            if lnt not in Q:
                continue
            old = len(Q)
            Q = union_lists(Q, right_non_terminals(r))
            if old != len(Q):
                changed = True
    return compact_grammar([r for r in g_ if left_non_terminals(r).pop() in Q])

In [13]:
new_grammar = delete_unreachable_non_terminals(g)

In [14]:
for r in new_grammar:
    print(r)

S -> AS | s
A -> a


# Удаление бесполезных символов

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B1%D0%B5%D1%81%D0%BF%D0%BE%D0%BB%D0%B5%D0%B7%D0%BD%D1%8B%D1%85_%D1%81%D0%B8%D0%BC%D0%B2%D0%BE%D0%BB%D0%BE%D0%B2_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

Нетерминал $A$ называется полезным (англ. useful) в КС-грамматике $\Gamma$, если он может участвовать в выводе, то есть существует порождение вида $S \Rightarrow ^* \alpha A \beta \Rightarrow ^* w$. Иначе он называется бесполезным (англ. useless).

Грамматика $\Gamma$ не содержит бесполезных нетерминалов тогда и только тогда, когда грамматика $\Gamma$ не содержит ни недостижимых нетерминалов, ни непорождающих.

Алгоритм состоит из двух этапов:

1. Удалить из грамматики правила, содержащие непорождающие нетерминалы.
2. Удалить из грамматики правила, содержащие недостижимые нетерминалы.

In [15]:
def delete_useless_symbols(g):
    g_ = delete_non_generating_rules(g)
    return delete_unreachable_non_terminals(g_)

In [16]:
g = [
    'S -> AS | BS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [17]:
new_grammar = delete_useless_symbols(g)

In [18]:
for r in new_grammar:
    print(r)

S -> AS | s
A -> a


# Удаление цепных правил

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%86%D0%B5%D0%BF%D0%BD%D1%8B%D1%85_%D0%BF%D1%80%D0%B0%D0%B2%D0%B8%D0%BB_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

__Цепная пара__ (unit pair) — упорядоченная пара (A,B), в которой $A\Rightarrow ^* B$, используя только цепные правила.

Алгоритм удаления цепных правил из грамматики:

1. Найти все цепные пары в грамматике $\Gamma$.
2. Для каждой цепной пары $(A,B)$ добавить в грамматику $\Gamma'$ все правила вида $A\rightarrow\alpha$, где $B\rightarrow\alpha$ — нецепное правило из $\Gamma$.
3. Удалить все цепные правила

In [19]:
g = [
    'A -> B | a', 
    'B -> C | b',
    'C -> DD | c'
] 

In [20]:
def delete_unit_rules(g):
    g_ = explode_grammar(g)
    pairs = [(k, k) for k in all_non_terminals(g)]
    
    to_delete = []
    for r in g_:
        lnt = left_non_terminals(r).pop()
        rnt = right_non_terminals(r)
        if len(rnt) > 1:
            continue
        for a, b in pairs:  
            if b == lnt:
                for x in rnt:
                    if (a, x) not in pairs:
                        to_delete += [r]
                        pairs += [(a, x)]

    pairs = [(a,b) for (a,b) in pairs if a != b] 
    
    new_rules = []
    for a,b in pairs:
        for r in g_:
            lnt = left_non_terminals(r).pop()
            if lnt != b:
                continue
            candidate = (a, right(r))
            if candidate not in pairs:
                new_rules += [a + ' -> ' + right(r)]
    
    new_rules.extend(g_)
    new_rules = [x for x in new_rules if x not in to_delete]
    
    return compact_grammar(new_rules)

In [21]:
new_grammar = delete_unit_rules(g)

In [22]:
for r in new_grammar:
    print(r)

A -> b | DD | c | a
B -> DD | c | b
C -> DD | c


# Поиск eps-нетерминалов

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_eps-%D0%BF%D1%80%D0%B0%D0%B2%D0%B8%D0%BB_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

Правила вида $A \to \varepsilon$ называются $\varepsilon$-правилами (англ. $\varepsilon$-rule).

Нетерминал $A$ называется $\varepsilon$-порождающим (англ. $\varepsilon$-generating), если $A \Rightarrow^* \varepsilon$.

Алгоритм поиска $\varepsilon$-порождающих нетерминалов

1. Найти все $\varepsilon$-правила. Составить множество, состоящее из нетерминалов, входящих в левые части таких правил.
2. Перебираем правила грамматики $\Gamma$. Если найдено правило $A \rightarrow C_1C_2...C_k$, для которого верно, что каждый $C_i$ принадлежит множеству, то добавить $A$ в множество.
3. Если на шаге 2 множество изменилось, то повторить шаг 2.

In [52]:
g = [
    'S -> ABC',
    'S -> DS',
    'A -> #',
    'B -> AC',
    'C -> #',
    'D -> d'
]

In [53]:
def find_esp_non_terminals(g):
    g_ = explode_grammar(g)
    Q = []
    changed = True
    while changed:
        changed = False
        for r in g_:
            all_non_terms_are_eps = is_subset_of(right_non_terminals(r), Q) and len(right_terminals(r)) == 0
            if right(r) == '#' or all_non_terms_are_eps:
                old = len(Q)
                Q = union_lists(Q, left_non_terminals(r))
                if old != len(Q):
                    changed = True
    return Q

In [54]:
Q = find_esp_non_terminals(g)

In [55]:
print(Q)

['C', 'B', 'S', 'A']


# Удаление eps-правил

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D0%B4%D0%B0%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_eps-%D0%BF%D1%80%D0%B0%D0%B2%D0%B8%D0%BB_%D0%B8%D0%B7_%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B8

Алгоритм удаления $\varepsilon$-правил из грамматики

1. Добавить все правила из $P$ в $P'$.
2. Найти все $\varepsilon$-порождаюшие нетерминалы.
3. Для каждого правила вида $A \rightarrow \alpha_0 B_1 \alpha_1 B_2 \alpha_2 ... B_k \alpha_k$ (где $\alpha_i$ — последовательности из терминалов и нетерминалов, $B_j$ — $\varepsilon$-порождающие нетерминалы) добавить в $P'$ все возможные варианты правил, в которых либо присутствует, либо удалён каждый из нетерминалов $B_j\; (1 \leqslant j \leqslant k)$.
4. Удалить все $\varepsilon$-правила из $P'$.
5. Если в исходной грамматике $\Gamma$ выводилось $\varepsilon$, то необходимо добавить новый нетерминал $S'$, сделать его стартовым, добавить правило $S' \rightarrow S|\varepsilon$.

# Устранение непосредственной левой рекурсии

https://neerc.ifmo.ru/wiki/index.php?title=%D0%A3%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BB%D0%B5%D0%B2%D0%BE%D0%B9_%D1%80%D0%B5%D0%BA%D1%83%D1%80%D1%81%D0%B8%D0%B8

Говорят, что контекстно-свободная (КС) грамматика $\Gamma$ содержит непосредственную левую рекурсию (англ. direct left recursion), если она содержит правило вида $A \to A\alpha$.

Методы нисходящего разбора не в состоянии работать с леворекурсивными грамматиками. Проблема в том, что продукция вида $A \Rightarrow^* A\alpha$ может применяться бесконечно долго, так и не выработав некий терминальный символ, который можно было бы сравнить со строкой. Поэтому требуется преобразование грамматики, которое бы устранило левую рекурсию.

Опишем процедуру, устраняющую все правила вида $A \to A\alpha$, для фиксированного нетерминала $A$.

1. Запишем все правила вывода из $A$ в виде:  $A \to A\alpha_1 \mid \ldots \mid A\alpha_n \mid \beta_1 \mid \ldots \mid \beta_m$, где
    * $\alpha$ — непустая последовательность терминалов и нетерминалов ($\alpha \nrightarrow \varepsilon$);
    * $\beta$ — непустая последовательность терминалов и нетерминалов, не начинающаяся с $A$.
    
2. Заменим правила вывода из $A$ на $A \to\beta_1A^\prime \mid \ldots\ \mid \beta_mA^\prime \mid \beta_1 \mid \ldots \mid \beta_m$.
3. Создадим новый нетерминал ${A^\prime} \to \alpha_1{A^\prime} \mid \ldots \mid \alpha_n{A^\prime} \mid \alpha_1 \mid \ldots \mid \alpha_n$.

Изначально нетерминал $A$ порождает строки вида $\beta\alpha_{i0}\alpha_{i1} \ldots \alpha_{ik}$. В новой грамматике нетерминал $A$ порождает $\beta{A^\prime}$, а $A^\prime$ порождает строки вида $\alpha_{i0}\alpha_{i1} \ldots \alpha_{ik}$. Из этого очевидно, что изначальная грамматика эквивалентна новой.